# Código utilizado para análise de dados da reportagem: [Prefeito e 5 vereadores eleitos em 2024 estão na lista suja do trabalho escravo](https://g1.globo.com/politica/noticia/2025/01/28/prefeito-e-5-vereadores-eleitos-em-2024-estao-na-lista-suja-do-trabalho-escravo.ghtml).
## Neste primeiro código, os dados de [sócios](https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/?C=N;O=D) foram cruzados com os dados das empresas que estão presentes na [Ficha Suja do trabalho escravo](https://www.gov.br/trabalho-e-emprego/pt-br/assuntos/inspecao-do-trabalho/areas-de-atuacao/cadastro_de_empregadores.pdf) para encontrar quem são os sócios autuados.

In [1]:
# Importa as bibliotecas necessárias para a análise
import pandas as pd
import os

In [2]:
# Me mostre tudo
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
# Acessando a pasta com dados de sócios
pasta = "dfs-receita/"

In [4]:
arquivos_csv = [os.path.join(pasta, f) for f in os.listdir(pasta) if f.endswith('.csv')]

In [5]:
# Seleciona colunas nas tabelas
colunas_interesse = [0,2,3] 

In [6]:
# Criando lista de dfs
dataframes = []

In [7]:
# Função para ler os dfs na pasta
for arquivo in arquivos_csv:
    print(f"Lendo arquivo: {arquivo}")
    try:
        # Ler apenas as colunas de interesse, usando o separador correto (;)
        for chunk in pd.read_csv(arquivo, sep=';', usecols=colunas_interesse, header=None, chunksize=100000, low_memory=False, encoding='latin-1'):
            dataframes.append(chunk)
    except Exception as e:
        print(f"Erro ao processar o arquivo {arquivo}: {e}")

Lendo arquivo: dfs-receita/K3241.K03200Y0.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y1.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y2.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y3.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y4.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y5.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y6.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y7.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y8.D41214.csv
Lendo arquivo: dfs-receita/K3241.K03200Y9.D41214.csv


In [8]:
df_completo = pd.concat(dataframes, ignore_index=True)

In [9]:
df_completo.head()

,0,2,3
0,34152447,ANTONIO PAULO NETO,***749831**
1,34152447,MAGALI MARIA DE ARAUJO DANTAS,***036931**
2,54218789,JAMES ONNIG TAMDJIAN,***753768**
3,71981070,FRANCISCO RASCAGLIA NETO,***027338**
4,71981070,SOLANGE NANI RASCAGLIA,***549588**


In [10]:
df_completo.columns = ['cnpj_basico', 'socio', 'cpf']

In [11]:
# Tratar CNPJ como string para evitar problemas de cruzamento
df_completo['cnpj_basico'] = df_completo['cnpj_basico'].astype(str)

In [12]:
df_completo.head()

,cnpj_basico,socio,cpf
0,34152447,ANTONIO PAULO NETO,***749831**
1,34152447,MAGALI MARIA DE ARAUJO DANTAS,***036931**
2,54218789,JAMES ONNIG TAMDJIAN,***753768**
3,71981070,FRANCISCO RASCAGLIA NETO,***027338**
4,71981070,SOLANGE NANI RASCAGLIA,***549588**


In [13]:
# Unindo com df de Ficha Suja do Trabalho Escravo
ficha = pd.read_csv("data/ficha.csv", delimiter =',', encoding='utf-8', dtype={"CNPJ/CPF": str})

In [14]:
ficha.head()

,Empregador,CNPJ/CPF
0,ABDIAS PEREIRA DA SILVA,126.420.376-49
1,ACACIO DE MELO FIGUEIREDO,969.417.078-87
2,ACADEMIA ACTTIONOLINDA LTDA,53.093.685/0001-24
3,ADALCIMAR DE OLIVEIRA LIMA,153.980.052-00
4,ADAN RAMON ALVARENGA CANDIA,35.702.272/0001-18


In [15]:
#Retriando sinais dos números do CNPJ
ficha['CNPJ/CPF'] = ficha['CNPJ/CPF'].str.replace(r'\D', '', regex=True)

In [16]:
# Filtrar apenas aqueles que tem CNPJs (14 caracteres) e retirandos CPFs
ficha_cnpj = ficha[ficha['CNPJ/CPF'].str.len() == 14]

In [17]:
ficha_cnpj.head()

,Empregador,CNPJ/CPF
2,ACADEMIA ACTTIONOLINDA LTDA,53093685000124
4,ADAN RAMON ALVARENGA CANDIA,35702272000118
15,AGROARACA INDUSTRIA DE ALIMENTOS LTDA,04239719000130
16,AGRO FORTE SERVICOS AGRICOLAS LTDA,34502408000183
17,AGROPECUARIA HARMONIA LTDA,34180456000100


In [18]:
# Criando tabela apenas com CPFs (MEIs)
ficha_cpf = ficha[ficha['CNPJ/CPF'].str.len() == 11]

In [19]:
ficha_cpf.head()

,Empregador,CNPJ/CPF
0,ABDIAS PEREIRA DA SILVA,12642037649
1,ACACIO DE MELO FIGUEIREDO,96941707887
3,ADALCIMAR DE OLIVEIRA LIMA,15398005200
5,ADAUTO VIANNA DINIZ,00875562000
6,ADEMAR BLOCH,24088811968


In [20]:
ficha_cnpj['CNPJ/CPF'] = ficha_cnpj['CNPJ/CPF'].str[:df_completo['cnpj_basico'].str.len().max()]

C:\Users\judite\AppData\Local\Temp\ipykernel_22204\3587951532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ficha_cnpj['CNPJ/CPF'] = ficha_cnpj['CNPJ/CPF'].str[:df_completo['cnpj_basico'].str.len().max()]


In [21]:
# Cruzando tabela de sócios com CNPJ com tabela de Ficha Suja do trabalho Escravo
merged_df = pd.merge(ficha_cnpj, df_completo, left_on='CNPJ/CPF', right_on='cnpj_basico', how='left')

In [22]:
merged_df.head()

,Empregador,CNPJ/CPF,cnpj_basico,socio,cpf
0,ACADEMIA ACTTIONOLINDA LTDA,53093685,53093685,RAFAEL VIEIRA DE LIMA,***307354**
1,ADAN RAMON ALVARENGA CANDIA,35702272,NaN,NaN,NaN
2,AGROARACA INDUSTRIA DE ALIMENTOS LTDA,04239719,NaN,NaN,NaN
3,AGRO FORTE SERVICOS AGRICOLAS LTDA,34502408,34502408,JOSE CARLOS OMODEI DA COSTA,***751829**
4,AGROPECUARIA HARMONIA LTDA,34180456,34180456,ALDROVANDI DE CARVALHO ROCHA,***891741**


In [23]:
# Cruzando tabela de sócios com MEI com tabela de Ficha Suja do trabalho Escravo
final_df = pd.concat([merged_df, ficha_cpf], ignore_index=True)

In [24]:
final_df

,Empregador,CNPJ/CPF,cnpj_basico,socio,cpf
0,ACADEMIA ACTTIONOLINDA LTDA,53093685,53093685,RAFAEL VIEIRA DE LIMA,***307354**
1,ADAN RAMON ALVARENGA CANDIA,35702272,NaN,NaN,NaN
2,AGROARACA INDUSTRIA DE ALIMENTOS LTDA,04239719,NaN,NaN,NaN
3,AGRO FORTE SERVICOS AGRICOLAS LTDA,34502408,34502408,JOSE CARLOS OMODEI DA COSTA,***751829**
4,AGROPECUARIA HARMONIA LTDA,34180456,34180456,ALDROVANDI DE CARVALHO ROCHA,***891741**
5,AGRO PECUARIA NOVA GALIA LTDA,52508116,52508116,RAYMOND BASSOULET,***164578**
6,AGRO PECUARIA NOVA GALIA LTDA,52508116,52508116,AGRO PECUARIA DOIS AMIGOS LTDA,51729267000155
7,AGROPECUARIA RIO ARATAU LTDA.,05078415,NaN,NaN,NaN
8,AJ CONSULTORIA AGROAMBIENTAL EIRELI,24241231,24241231,ADELSON NEVES DA SILVA,***004921**
9,AL E C INSTALACOES E MANUTENCOES LTDA,38418949,38418949,AUGUSTO NERI DE SOUZA SAMPAIO NETO,***390505**


In [25]:
# Salvar o resultado em um arquivo csv
merged_df.to_csv('data/socios-fichasuja.csv', index=False)